In [3]:
!pip install pandas 
!pip install psycopg2-binary


[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
  Using cached psycopg2_binary-2.9.9-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)

[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)

[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
!pip install sqlalchemy

  Using cached SQLAlchemy-2.0.31-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)

[notice] A new release of pip available: 22.2.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import logging
import requests
import pandas as pd
from kafka import KafkaProducer
import json
import psycopg2
from sqlalchemy import create_engine, text
from scipy.special import boxcox
import numpy as np
import uuid
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [3]:
def zscore_normalization(df, name, preprocess_info):
    pair = preprocess_info[name.lower()]
    df[name] = (df[name] - pair[0]) / pair[1]
    
def preprocess(df):
    import json
    with engine.connect() as conn: 
        query = text('SELECT * FROM metadata_table_malware ORDER BY version DESC LIMIT 1')
        data = pd.read_sql_query(query, conn)
        row = data.iloc[0]
        factors = row['factor']
    df = df.drop(columns=['Name', 'md5'])
    for i in df.columns:
        if i != 'legitimate' and factors[i.lower()] == None:
            df.drop(columns=i, inplace=True)
        elif i != 'legitimate':
            df[i] = boxcox(df[i], 0.5)
            zscore_normalization(df, i, factors)
    return df

In [4]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret():

    secret_name = "DBCreds"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    # Parse the secret string to get the credentials
    secret_dict = json.loads(secret)
    username = secret_dict['username']
    password = secret_dict['password']
    host = secret_dict['host']
    port = secret_dict['port']
    dbname = secret_dict['dbname']

    return username, password, host, port, dbname


(user,pswd,host,port,db) = get_secret()

In [6]:
# Initialize logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Broker address
brokers = [f'{host}:9092']

# GitHub raw URL for the CSV file
github_url = "https://raw.githubusercontent.com/tsimhadri-ews/internproject/malware-detection-0/src/MalwareData.csv"

# PostgreSQL database connection details
db_config = {
    'dbname': db,
    'user': user,
    'password': pswd,
    'host': host,
    'port': port
}

# PostgreSQL connection setup
logger.info("Connecting to PostgreSQL...")
try:
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")
    logger.info("PostgreSQL connection established successfully.")
except Exception as e:
    logger.error(f"Failed to connect to PostgreSQL: {e}")
    exit()

# Kafka topics
postgres_topic = "postgresql"
kserve_topic = "three"

# Download the CSV file from GitHub
logger.info("Downloading CSV file from GitHub...")
response = requests.get(github_url)

# Check if the request was successful
if response.status_code == 200:
    logger.info("CSV file downloaded successfully.")
    # Read the data into a pandas DataFrame
    df = pd.read_csv(github_url, sep='|')
    unprocess_df = df
    raw_df = df
    df = preprocess(df)
    raw_df = raw_df[df.columns]
    raw_df = raw_df.drop(columns=['legitimate'])
    df = df.drop(columns=['legitimate'])
    logger.info("CSV file processed successfully.")
else:
    logger.error(f"Failed to download file: {response.status_code}")
    exit()
df['uid'] = [str(uuid.uuid4()) for _ in range(len(df))]
unprocess_df['uid'] = [str(uuid.uuid4()) for _ in range(len(unprocess_df))]
# Create Kafka producers
logger.info("Creating Kafka producers...")
postgres_producer = KafkaProducer(
    bootstrap_servers=brokers,
    value_serializer=lambda message: json.dumps({k: str(v).lower() for k, v in message.items()}).encode('utf-8'),
)

kserve_producer = KafkaProducer(
    bootstrap_servers=brokers,
    value_serializer=lambda message: json.dumps(message).encode('utf-8'),
)

# Send data to KServe and PostgreSQL
first_5_rows = df.head(5)
for index, row in first_5_rows.iterrows():
    data = row.to_dict()
    # Send to KServe topic
    kserve_producer.send(kserve_topic, data)
    logger.info(f"Sent row {index + 1} to KServe Kafka topic")

kserve_producer.flush()
kserve_producer.close()
# Flush data and close the producers

first_5_unprocess = unprocess_df.head(5)
for index, row in first_5_unprocess.iterrows():
    data = row.to_dict()
    # Send to PostgreSQL topic
    postgres_producer.send(postgres_topic, data)
    logger.info(f"Sent row {index + 1} to PostgreSQL Kafka topic")
# Flush data and close the producers
postgres_producer.flush()
postgres_producer.close()

logger.info(f"First 5 rows sent to PostgreSQL Kafka topic: {postgres_topic} and KServe Kafka topic: {kserve_topic}")


INFO:root:Connecting to PostgreSQL...
INFO:root:PostgreSQL connection established successfully.
INFO:root:Downloading CSV file from GitHub...
INFO:root:CSV file downloaded successfully.
INFO:root:CSV file processed successfully.
INFO:root:Creating Kafka producers...
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=54.224.168.37:9092 <connecting> [IPv4 ('54.224.168.37', 9092)]>: connecting to 54.224.168.37:9092 [('54.224.168.37', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=54.224.168.37:9092 <connecting> [IPv4 ('54.224.168.37', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=54.224.168.37:9092 <connecting> [IPv4 ('54.224.168.37', 9092)]>: connecting to 54.224.168.37:9092 [('54.224.168.37', 9092) IPv4]
IN